In [1]:
from utils import *
from variables import *
from datetime import datetime, timedelta
import requests
import re
from scipy.stats import norm
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pyhtml2pdf import  converter
import shutil
from plotly.colors import n_colors
from typing import Union, List, Dict
from bs4 import BeautifulSoup
import traceback
from utils import parse_excel
from generate_files import generate_pdfs, generate_parquets
from openpyxl.reader.excel import load_workbook
from openpyxl.utils.cell import coordinate_to_tuple

In [ ]:
def get_contracts_excel(auswahl : int, excel_path : str):

    dict_auswahl_prefix = {
        0 : "",
        1 : "STOXX_"
    }

    wb = load_workbook(excel_path, data_only=True, keep_vba=True)
    Ueberhaenge_sheet = wb['Ueberhaenge']
    heute = Ueberhaenge_sheet.cell(*coordinate_to_tuple("G5")).value
    expiry = Ueberhaenge_sheet.cell(*coordinate_to_tuple("G3")).value
    expiry_1 = Ueberhaenge_sheet.cell(*coordinate_to_tuple("R6")).value
    stock_price = Ueberhaenge_sheet.cell(*coordinate_to_tuple("X4")).value
    InterestRate = Ueberhaenge_sheet.cell(*coordinate_to_tuple("N3")).value
    ZentralKurs =  Ueberhaenge_sheet.cell(*coordinate_to_tuple("C3")).value  
    volatility = Ueberhaenge_sheet.cell(*coordinate_to_tuple("N4")).value

    Summery_sheet = wb[f'{dict_auswahl_prefix[auswahl]}Summery']
    nbd_heute = Summery_sheet.cell(*coordinate_to_tuple("C9")).value
   
    nbd_last = Summery_sheet.cell(*coordinate_to_tuple("C9")).value
    nbd_last = next_business_day(heute)
    nbd_dict = {'heute' : nbd_heute, 'last' : nbd_last}


    tage_bis_verfall = (expiry - heute).days #+ 1

    print(f"tage_bis_verfall = {tage_bis_verfall}")
    print(f"stock_price = {stock_price}")
    print(f"InterestRate = {InterestRate}")
    print(f"volatility = {volatility}")
    print(f"InterestRate = {InterestRate}")


    dict_prod_bus = {}
    for productdate_idx in [0,1]:
        dict_prod_bus[productdate_idx] = {}
        for busdate_idx in [0,1]:
            print(f"busdate_idx = {busdate_idx}   | productdate_idx = {productdate_idx}")
            if (busdate_idx == 1) and (productdate_idx == 1):
                # No need to request data for this case so we skip this iteration
                break

            if (busdate_idx == 0) and (productdate_idx == 0):
                contractsCall_aux_df =  pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}Call_Front")
                contractsPut_aux_df =  pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}Put_Front")

            if (busdate_idx == 1) and (productdate_idx == 0):
                contractsCall_aux_df= pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}CallFront-1")
                contractsPut_aux_df = pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}PutFront-1")

            if (busdate_idx == 0) and (productdate_idx == 1):
                contractsCall_aux_df = pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}Put+01")
                contractsPut_aux_df= pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}Call+01")


            aux_df = contractsCall_aux_df[['strike','openInterest']].merge(
                contractsPut_aux_df[['strike','openInterest']],
                on = "strike",
                how="left",
                suffixes=('_CF', '_PF')
            )
            dict_prod_bus[productdate_idx][busdate_idx] = aux_df
    return auswahl, heute, expiry, expiry_1, stock_price, InterestRate, volatility, ZentralKurs, tage_bis_verfall, nbd_dict, dict_prod_bus

In [8]:
auswahl, heute, expiry, expiry_1, stock_price, InterestRate, volatility, ZentralKurs, tage_bis_verfall, nbd_dict, dict_prod_bus, list_email_send_selection, Summery_df, HedgeBedarf_df, HedgeBedarf1_df, delta, future_date_col = parse_eurex(0)

heute = 13/09/2023
################## Obtaining overview ##################
DAX
Getting data from EUREX...
try = 0
################## Obtaining contracts ##################
busdate_idx = 0   | productdate_idx = 0 | try = 0
busdate_idx = 1   | productdate_idx = 0 | try = 0
busdate_idx = 0   | productdate_idx = 1 | try = 0
busdate_idx = 0   | productdate_idx = 1 | try = 1
busdate_idx = 1   | productdate_idx = 1 | try = 0
tage_bis_verfall = 2
stock_price = 15653.5
Minkurs = 14500.0
Maxkurs = 16500.0
Schritte = 40
ZentralKurs = 15500.0
volatility_Laufzeit = 60
Euribor 3M in 9/12/2023 is 3.824 %
vdax-new-2m = 15.49 PTS
volatility = 0.1549
InterestRate = 0.038239999999999996
Excel files have been exported.


In [2]:
auswahl, heute, expiry, expiry_1, stock_price, InterestRate, volatility, ZentralKurs, tage_bis_verfall, nbd_dict, dict_prod_bus = parse_excel(0, "./excel_files/EUREX 2023-08-28.xlsm")

c:\Users\Vinicius\anaconda3\envs\eurex\Lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Vinicius\anaconda3\envs\eurex\Lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


tage_bis_verfall = 18
stock_price = 15683
InterestRate = 0.03784
volatility = 0.17079999999999998
InterestRate = 0.03784
busdate_idx = 0   | productdate_idx = 0
busdate_idx = 1   | productdate_idx = 0
busdate_idx = 0   | productdate_idx = 1
busdate_idx = 1   | productdate_idx = 1


In [12]:
print(f"heute = {heute}")
# Create results folder in case they still were not created
list_folder_results =  [current_results_path, old_results_path, temp_results_path]
for folder in list_folder_results:
    create_folder(folder)

if auswahl == 0: # DAX
    Spannweite = 2000
    Schritt = 50
    volatility_Laufzeit = 60
    KontraktWert = 5

else: # STOXX
    Spannweite = 700
    Schritt = 25
    volatility_Laufzeit = 365
    KontraktWert = 1

span = (Spannweite/4)

Minkurs = ZentralKurs - (Spannweite/ 2)

Maxkurs = Minkurs + Spannweite
Schritte = int(Spannweite / Schritt)

DetailMin = round(stock_price - (Spannweite)/4)
DetailMax = DetailMin + (Spannweite/2)


if tage_bis_verfall >= 1:
    delta = 0.5
else:
    delta = 1

print(f"stock_price = {stock_price}")
print(f"Minkurs = {Minkurs}")
print(f"Maxkurs = {Maxkurs}")
print(f"Schritte = {Schritte}")
print(f"ZentralKurs = {ZentralKurs}")
print(f"volatility_Laufzeit = {volatility_Laufzeit}")




# Second stage
# Create Basis column that will be used for multiple tables
Basis = pd.Series((Minkurs + np.arange(int(Schritte) +1)* Schritt)[::-1])

# Transform series into dataframe to make dataframe creation easier 
Basis_df = Basis.reset_index().rename(columns={0:"Basis"})[["Basis"]]


# Ueberhaenge_df, Summery_df are created from a copy of the same Dataframe
# so they have the same index. It means they could be in one  Dataframe but
#  I decided to keep them separated as they were in the VBA code.

Ueberhaenge_df = Basis_df.copy()
Summery_df = Basis_df.copy()

for productdate_idx in [0,1]:
    for busdate_idx in [0,1]:

        if (busdate_idx == 1) and (productdate_idx == 1):
            continue

        # Create dataframe with the just requested data
        aux_df = Basis_df.copy()

        aux_df = aux_df.merge(
            dict_prod_bus[productdate_idx][busdate_idx],
            left_on = "Basis",
            right_on = "strike",
            how= "left"
        )

        if productdate_idx == 0:
            Ueberhaenge_df[busdate_idx] = aux_df.openInterest_PF - aux_df.openInterest_CF

            if busdate_idx == 0:
                Summery_df["openInterest_PF"] = aux_df["openInterest_PF"]
                Summery_df["openInterest_CF"] = aux_df["openInterest_CF"]
        
        elif productdate_idx == 1:
            if busdate_idx == 0:
                Ueberhaenge_df["nextContract"] = aux_df.openInterest_PF - aux_df.openInterest_CF
                Summery_df["nextContract"] = Ueberhaenge_df["nextContract"]


Ueberhaenge_df.rename(columns = {0 : "Front"},inplace=True)
Ueberhaenge_df.rename(columns = {1 : "Last"},inplace=True)


Summery_df["heute"] = Ueberhaenge_df["Front"] * (1 / KontraktWert) * delta
Summery_df["last_day"] = Ueberhaenge_df["Last"] * (1 / KontraktWert) * delta

Ueberhaenge_df["Summe"] = Ueberhaenge_df[["Front", "nextContract"]].sum(axis=1)
Ueberhaenge_df = Ueberhaenge_df[['Basis', 'Summe',"Last",'Front', "nextContract"]]


Summery_df["Änderung"] = Summery_df.heute - Summery_df.last_day

if (delta == 1):
    Summery_df['nextContract'] =  Summery_df['nextContract'] / 2

SummeryDetail_df = Summery_df[(Summery_df.Basis >= DetailMin) & (Summery_df.Basis < (DetailMax + Schritt))]

# Third stage
SchrittWeite = 10

heute = 2023-09-11 00:00:00
stock_price = 4241.34
Minkurs = 3950.0
Maxkurs = 4650.0
Schritte = 28
ZentralKurs = 4300
volatility_Laufzeit = 365


In [13]:
Tage = tage_bis_verfall
if Tage == 0:
    Tage = 0.5

Tage_1 = (expiry_1 - heute).days +1
Kurs_count = int(Spannweite/SchrittWeite)+1 
HedgeBedarf_kurs=  pd.DataFrame(Maxkurs - np.arange(Kurs_count)* SchrittWeite,columns= ["Basis"])


Hedge_dimensions = Kurs_count,int(Schritte+1)
HedgeBedarf_values = np.zeros(Hedge_dimensions)
HedgeBedarf1_values = np.zeros(Hedge_dimensions)


for k in range(Hedge_dimensions[1]):
    Basis_value = Basis[k]
    Kontrakte = Ueberhaenge_df.loc[k,"Front"]
    Kontrakte_1 = Ueberhaenge_df.loc[k,"nextContract"]
    for i in range(Hedge_dimensions[0]):
        Kurs = Maxkurs - i * SchrittWeite
        #print(Kurs)
        # In python np.log = natural log
        h1 = np.log(Kurs / Basis_value)
        if auswahl == 0:
            sigma = volatility * ((Tage / volatility_Laufzeit) ** 0.5)
        else:
            sigma = volatility

        sigma_1 = volatility * ((Tage_1 / volatility_Laufzeit) ** 0.5)
        h2 = InterestRate + sigma * sigma / 2
        h2_1 = InterestRate + sigma_1 * sigma_1 / 2
        d1 = (h1 + (h2 * (Tage / 365))) / (sigma * ((Tage / 365) ** 0.5))
        d1_1 = (h1 + (h2_1 * (Tage_1 / 365))) / (sigma_1 * ((Tage_1 / 365) ** 0.5))
        Phi = norm.pdf(d1, 0, 1)               
        Phi_1 = norm.pdf(d1_1 + 0.01, 0, 1)
        Gamma = Phi / (Kurs * (sigma * (Tage / 365) ** 0.5))
        Gamma_1 = Phi_1 / (Kurs * (sigma_1 * (Tage_1 / 365) ** 0.5))
        
        HedgeBedarf_values[i,k] = Gamma * Kontrakte / KontraktWert
        HedgeBedarf1_values[i,k] = Gamma_1 * Kontrakte_1 / KontraktWert


HedgeSum = HedgeBedarf_values.sum(axis=1)/2
HedgeSum_1 = HedgeBedarf1_values.sum(axis=1)/2

HedgeBedarf_df = pd.DataFrame(data =HedgeBedarf_values, columns= Basis )
HedgeSum_df = pd.DataFrame(HedgeSum,columns=["HedgeSum"])
HedgeBedarf_df = pd.concat([HedgeBedarf_kurs,HedgeSum_df,HedgeBedarf_df], axis=1)

HedgeBedarf1_df = pd.DataFrame(data =HedgeBedarf1_values, columns= Basis)
HedgeSum1_df = pd.DataFrame(HedgeSum_1,columns=["HedgeSum"])
HedgeBedarf1_df = pd.concat([HedgeBedarf_kurs,HedgeSum1_df,HedgeBedarf1_df], axis=1)



In [14]:
HedgeBedarf_excel = pd.read_excel(excel_path, sheet_name = f"{dict_auswahl_prefix[auswahl]}HedgeBedarf",skiprows=3).rename(columns = {"Unnamed: 0" : "Basis", "Unnamed: 2" : "HedgeSum"})[['Basis', 'HedgeSum']]
HedgeBedarf_excel.head()

,Basis,HedgeSum
0,4650,-860.492729
1,4640,-925.722654
2,4630,-986.574975
3,4620,-1042.048037
4,4610,-1091.308225


In [15]:
compare_df = HedgeBedarf_df[['Basis', 'HedgeSum']].merge(HedgeBedarf_excel, on = 'Basis',suffixes=('_py', '_excel'))

# Biggest difference between python and excel
(compare_df.HedgeSum_py - compare_df.HedgeSum_excel).abs().max()


compare_df['dif'] = (compare_df.HedgeSum_py - compare_df.HedgeSum_excel).abs()
compare_df.loc[compare_df.dif.sort_values(ascending=False).index].head(20)

,Basis,HedgeSum_py,HedgeSum_excel,dif
5,4600.0,-1133.700483,-1133.700483,4.547474e-13
14,4510.0,-1169.051749,-1169.051749,4.547474e-13
9,4560.0,-1227.229740,-1227.229740,4.547474e-13
2,4630.0,-986.574975,-986.574975,3.410605e-13
24,4410.0,-627.559073,-627.559073,3.410605e-13
1,4640.0,-925.722654,-925.722654,3.410605e-13
64,4010.0,1225.148343,1225.148343,2.273737e-13
12,4530.0,-1214.905338,-1214.905338,2.273737e-13
62,4030.0,1300.381536,1300.381536,2.273737e-13
15,4500.0,-1135.670885,-1135.670885,2.273737e-13


In [3]:
#auswahl = 1
#excel_path = "./excel_files/STOXX_OI-Abfrage 2023-09-11 V9.00.xlsm"

auswahl = 0
excel_path = "./excel_files/EUREX 2023-08-28.xlsm"

Summery_df = pd.read_parquet(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_Summery_df.parquet'))
HedgeBedarf_df = pd.read_parquet(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_HedgeBedarf_df.parquet'))
HedgeBedarf1_df = pd.read_parquet(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_HedgeBedarf1_df.parquet'))

stock_price = read_pickle(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_stock_price.pickle'))
heute = read_pickle(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_heute.pickle'))
tradingDates = read_pickle(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_tradingDates.pickle'))
future_date_col = read_pickle(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_future_date_col.pickle'))

expiry = read_pickle(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_expiry.pickle'))
tage_bis_verfall = read_pickle(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_tage_bis_verfall.pickle'))
delta = read_pickle(os.path.join(temp_results_path, f'{dict_index_stock[auswahl]}_delta.pickle'))    

In [3]:
# Fifth stage
# Values to create arrow
idx_closest = (HedgeBedarf_df.Basis - stock_price).abs().idxmin()
closest_Basis =  HedgeBedarf_df.loc[idx_closest,"Basis"]
closest_Sum = HedgeBedarf_df.loc[idx_closest,"Sum"]

x_axis_length = HedgeBedarf_df.Sum.max() - HedgeBedarf_df.Sum.min()
y_axis_length = HedgeBedarf_df.Basis.max() - HedgeBedarf_df.Basis.min()

min_Kontrakte = 5000
max_Differenz = 1000
prozentual = 0.2

condition_close_verfall = (auswahl == 0) and tage_bis_verfall < 5

if condition_close_verfall:
    indexes = Summery_df.loc[(Summery_df.Basis >= stock_price)].tail(10).index.to_list() + Summery_df.loc[(Summery_df.Basis < stock_price)].head(10).index.to_list()
    Summery_df =  Summery_df.loc[indexes].reset_index(drop=True)

    indexes = HedgeBedarf_df.loc[(HedgeBedarf_df.Basis >= Summery_df.Basis.min()) & (HedgeBedarf_df.Basis <= Summery_df.Basis.max())].index
    
    HedgeBedarf_df = HedgeBedarf_df.loc[indexes].reset_index(drop=True)
    HedgeBedarf1_df = HedgeBedarf1_df.loc[indexes].reset_index(drop=True)

# (CF > min_Kontrakte) AND (PF > min_Kontrakte) AND (ABS(CF - PF)  < MIN(PF,CF)*)
mask_highlights = (
    (Summery_df.openInterest_CF > min_Kontrakte) & 
    (Summery_df.openInterest_PF > min_Kontrakte) & 
    (
        (Summery_df.openInterest_CF - Summery_df.openInterest_PF).abs() < 
        (Summery_df[['openInterest_PF',"openInterest_CF"]].min(axis=1)*prozentual)))


################################## Hilights for Basis column ####################################

Summery_df["basis_color"] = "lavender"
Summery_df.loc[
    mask_highlights, "basis_color"] = "yellow"
col_basis_color = Summery_df.basis_color.to_numpy()
#################################################################################################

######################### Gradient of red and blue for Anderung column ##########################
aux = Summery_df.Änderung.reset_index()
aux['r'] = aux['g']  = aux['b'] = 255
aux.loc[aux.Änderung<0,'g'] = aux.loc[aux.Änderung<0,'b'] = 255 - 150*aux.Änderung/(aux.Änderung.min()) 
aux.loc[aux.Änderung>0,'g'] = aux.loc[aux.Änderung>0,'r'] = 255 - 150*aux.Änderung/(aux.Änderung.max())
aux = aux.astype(float)
rb_shades = np.array([f"rgb({aux.loc[i,'r']},{aux.loc[i,'g']},{aux.loc[i,'b']})" for i in range(aux.shape[0]) ])
col_anderung_color = rb_shades
#################################################################################################

############################# Colors for heute and last_day columns #############################
col_heute_color = posneg_binary_color(Summery_df.heute,"rgb(0, 204, 204)","rgb(77, 166, 255)")
col_last_day_color = posneg_binary_color(Summery_df.last_day,"rgb(0, 204, 204)","rgb(77, 166, 255)")
#################################################################################################

###################### Gradient of green and blue for Put anc Call columns ######################
col_put_color  = np.array(n_colors('rgb(214, 245, 214)', 'rgb(40, 164, 40)',
    20, colortype='rgb'))[scale_col_range(Summery_df.openInterest_PF,19)]

col_call_color  = np.array(n_colors('rgb(204, 224, 255)', 'rgb(0, 90, 179)',
    20, colortype='rgb'))[scale_col_range(Summery_df.openInterest_CF,19)]
#################################################################################################

num_rows = Summery_df.shape[0]+1
height = 1050 #row_height*42
row_height = int(height/(num_rows))

# width of the image
widht = 1000
row_height_percent =  (row_height/height)



values_body = [
            (Summery_df.Basis),
            (Summery_df.Änderung.round(0)),
            (Summery_df.heute.round(2)),
            (Summery_df.last_day.round(2)),

        ]
values_header = bold(["Basis","Änderung",next_business_day(tradingDates[0]).strftime("%d/%m/%y"),next_business_day(tradingDates[1]).strftime("%d/%m/%y")])


if not  condition_close_verfall:
    values_body += [(Summery_df.openInterest_PF), (Summery_df.openInterest_CF)]
    values_header += bold(["Put","Call"])


font_size =  int(440/num_rows)

fig = go.Figure(
    data=[
        go.Table(

            # Define some paremeters for the header
            header=dict(
                # Names of the columns
                values= values_header,
                
                # Header style
                fill_color='paleturquoise',
                    align='center',
                    font = {'size': int(font_size*0.8)},
                    height = row_height,
            ),

            cells=dict(

                align='center',
                height = row_height,
                font = {'size': font_size,},

                # Values of the table
                values= values_body,

                # Colors of the columns
                fill_color = [
                    col_basis_color,
                    col_anderung_color,
                    col_heute_color,
                    col_last_day_color,
                    col_put_color, 
                    col_call_color
                ],
            )
        )
    
    ],

)
fig.update_layout(
    height=1050, width=550,
    margin=dict(l=0,r=0,b=0.0,t=0)
)

fig.write_image(os.path.join(current_results_path,'teste1.svg'),scale=1)

fig.show()


data =  [
    go.Scatter(
        x = HedgeBedarf_df.Sum,
        y = HedgeBedarf_df.Basis,
        mode = "lines",
        name = datetime.strptime(future_date_col[0],"%Y%m%d").strftime("%Y-%m") + " + "+ datetime.strptime(future_date_col[1],"%Y%m%d").strftime("%Y-%m"),
        marker_color= "blue"
    )
]



if not condition_close_verfall:
    data += [
        go.Scatter(
            x = HedgeBedarf1_df.HedgeSum,
            y = HedgeBedarf_df.Basis,
            mode = "lines",
            name = datetime.strptime(future_date_col[1],"%Y%m%d").strftime("%Y-%m"),
            marker_color= "purple"
        ),
        go.Scatter(
        x = HedgeBedarf_df.HedgeSum,
        y = HedgeBedarf_df.Basis,
        mode = "lines",
        name = datetime.strptime(future_date_col[0],"%Y%m%d").strftime("%Y-%m"),
        marker_color= "orange"
        )
    ]

##################################################################################################
dx =  0.1*(HedgeBedarf_df.Sum.max() - HedgeBedarf_df.Sum.min())



fig = go.Figure(data=data)

# Create arrow
fig.add_trace(go.Scatter(
        x = [closest_Sum + x_axis_length/5,closest_Sum + x_axis_length/50], 
        y = [closest_Basis,closest_Basis],
    marker= dict(size=20,symbol= "arrow-bar-up", angleref="previous"),
    showlegend = False
    )
)


fig.update_layout(
    margin=dict(l=0,r=0,b=0.1,t=row_height),
    # Set limits in the x and y axis
    yaxis_range= [HedgeBedarf_df.Basis.min(), HedgeBedarf_df.Basis.max()],
    xaxis_range= [HedgeBedarf_df.Sum.min() - dx, HedgeBedarf_df.Sum.max() + dx],
    yaxis = dict(
        tickmode = 'array',
        tickvals = HedgeBedarf_df.Basis[HedgeBedarf_df.index % 3 == 0],
        #ticktext = ['One', 'Three', 'Five', 'Seven', 'Nine', 'Eleven']
    ),
    
    # Remove margins
    paper_bgcolor="white",
    
    # Define width and height of the image
    width=380,height=1050,
    template = "seaborn",

    # Legend parameters
    legend=dict(
        yanchor="top",
        y=1,# - (row_height/height),
        xanchor="right",
        x= 1,
        font=dict(
            size = 8
        ),
        # legend in the vertical
        orientation = "v",
        bgcolor  = 'rgba(0,0,0,0)'
        ),
)


fig.write_image(os.path.join(current_results_path,'teste2.svg'),scale=1)
fig.show()


fig = go.Figure()

if not condition_close_verfall:
    fig.add_trace(
        trace = go.Bar(name='Put', y=Summery_df.Basis, x=Summery_df.openInterest_PF,orientation='h', marker_color = 'rgb(40, 164, 40)'),
    )

    fig.add_trace(
        trace = go.Bar(name='Call', y=Summery_df.Basis, x=Summery_df.openInterest_CF,orientation='h', marker_color = 'rgb(0, 90, 179)'),
    )



fig.update_layout(
    barmode='stack', margin=dict(l=0,r=0,b=0.1,t=row_height),
    # Set limits in the x and y axis
    #yaxis_range= [HedgeBedarf_df.Basis.min(), HedgeBedarf_df.Basis.max()],
    #xaxis_range= [HedgeBedarf_df.Sum.min() - dx, HedgeBedarf_df.Sum.max() + dx],
    
    # Define width and height of the image
    width=80,height=1050,
    template = "seaborn",
    showlegend=False,
    bargap =0.5 ,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
)


fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)

fig.write_image(os.path.join(current_results_path,'teste3.svg'),scale=1)
fig.show()

NameError: name 'HedgeBedarf_df' is not defined

In [46]:
condition_close_verfall

True

In [25]:
stock_price

15735.5